In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline

In [14]:
analysis = Analysis.objects.get_or_create(id=2)[0]

DEBUG 2017-03-28 13:04:22,617 utils.py:91 | (0.001) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 2 LIMIT 21; args=(2,)


In [15]:
project = Project.objects.get_or_create(id=2)[0]

DEBUG 2017-03-28 13:04:24,788 utils.py:91 | (0.001) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 2 LIMIT 21; args=(2,)


In [16]:
comparisons = analysis.experiment.comparison_set.all()

DEBUG 2017-03-28 13:04:26,865 utils.py:91 | (0.001) SELECT `experiments_experiment`.`id`, `experiments_experiment`.`title`, `experiments_experiment`.`created`, `experiments_experiment`.`modified` FROM `experiments_experiment` WHERE `experiments_experiment`.`id` = 2 LIMIT 21; args=(2,)


In [17]:
print comparisons

DEBUG 2017-03-28 13:04:29,679 utils.py:91 | (0.001) SELECT `experiments_comparison`.`id`, `experiments_comparison`.`name`, `experiments_comparison`.`description`, `experiments_comparison`.`experiment_id` FROM `experiments_comparison` WHERE `experiments_comparison`.`experiment_id` = 2 LIMIT 21; args=(2,)


[<Comparison: comparison_1>, <Comparison: comparison_2>]


In [56]:
from collections import OrderedDict
def get_samples(comparisons):
    s = Sample.objects.filter(
        attribute=Attribute.objects.filter(comparison=comparisons).distinct().order_by('id')).distinct().order_by(
        'attribute__id', 'id')

    member_list = list(Attribute.objects.filter(comparison=comparisons).distinct().order_by('id'))
    print 'Member list: %s' % member_list

    sample_list = []
    for member in member_list:
        sample_list.extend(list(member.sample.all().order_by('id')))
    sample_list = list(OrderedDict.fromkeys(sample_list))
    print "sample list: %s" % sample_list

    return  sample_list

print get_samples(comparisons)

DEBUG 2017-03-28 14:24:51,125 utils.py:91 | (0.001) SELECT DISTINCT `groups_attribute`.`id`, `groups_attribute`.`name`, `groups_attribute`.`group_id`, `groups_attribute`.`created`, `groups_attribute`.`modified`, `groups_attribute`.`ticgroup_id` FROM `groups_attribute` INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) WHERE (`experiments_attributecomparison`.`comparison_id`) IN (SELECT U0.`id` FROM `experiments_comparison` U0 WHERE U0.`experiment_id` = 2) ORDER BY `groups_attribute`.`id` ASC; args=(2,)
DEBUG 2017-03-28 14:24:51,129 utils.py:91 | (0.001) SELECT `groups_group`.`id`, `groups_group`.`name`, `groups_group`.`created`, `groups_group`.`modified` FROM `groups_group` WHERE `groups_group`.`id` = 6 LIMIT 21; args=(6,)
DEBUG 2017-03-28 14:24:51,132 utils.py:91 | (0.001) SELECT `groups_group`.`id`, `groups_group`.`name`, `groups_group`.`created`, `groups_group`.`modified` FROM `groups_group` WHERE `groups_gr

Member list: [<Attribute: C4 (TimeCourse)>, <Attribute: C9 (TimeCourse)>, <Attribute: C24 (TimeCourse)>]
sample list: [<Sample: C1_4.mzXML>, <Sample: C2_4.mzXML>, <Sample: C3_4.mzXML>, <Sample: C1_9.mzXML>, <Sample: C2_9.mzXML>, <Sample: C3_9.mzXML>, <Sample: C1_24.mzXML>, <Sample: C2_24.mzXML>, <Sample: C3_24.mzXML>]
[<Sample: C1_4.mzXML>, <Sample: C2_4.mzXML>, <Sample: C3_4.mzXML>, <Sample: C1_9.mzXML>, <Sample: C2_9.mzXML>, <Sample: C3_9.mzXML>, <Sample: C1_24.mzXML>, <Sample: C2_24.mzXML>, <Sample: C3_24.mzXML>]


In [18]:
from fileupload.models import Sample

In [19]:
samples = Sample.objects.filter(attribute__comparison__in=comparisons).distinct().order_by('id')

In [20]:
for s in samples:
    print s.id, s.name

DEBUG 2017-03-28 13:04:38,093 utils.py:91 | (0.001) SELECT DISTINCT `fileupload_sample`.`id`, `fileupload_sample`.`name`, `fileupload_sample`.`project_id`, `fileupload_sample`.`samplefile_id` FROM `fileupload_sample` INNER JOIN `groups_sampleattribute` ON ( `fileupload_sample`.`id` = `groups_sampleattribute`.`sample_id` ) INNER JOIN `groups_attribute` ON ( `groups_sampleattribute`.`attribute_id` = `groups_attribute`.`id` ) INNER JOIN `experiments_attributecomparison` ON ( `groups_attribute`.`id` = `experiments_attributecomparison`.`attribute_id` ) WHERE (`experiments_attributecomparison`.`comparison_id`) IN (SELECT U0.`id` FROM `experiments_comparison` U0 WHERE U0.`experiment_id` = 2) ORDER BY `fileupload_sample`.`id` ASC; args=(2,)


13 C1_4.mzXML
14 C1_9.mzXML
15 C1_24.mzXML
16 C2_4.mzXML
17 C2_9.mzXML
18 C2_24.mzXML
19 C3_4.mzXML
20 C3_9.mzXML
21 C3_24.mzXML


In [61]:
sample_map = [sample.id for sample in samples]

In [65]:
sample_map

[13L, 16L, 19L, 14L, 17L, 20L, 15L, 18L, 21L]

In [64]:
sample_map = [x.id for x in sorted(samples, key=lambda x: x.attribute_set.first().id)]

DEBUG 2017-03-28 14:30:56,645 utils.py:91 | (0.001) SELECT `groups_attribute`.`id`, `groups_attribute`.`name`, `groups_attribute`.`group_id`, `groups_attribute`.`created`, `groups_attribute`.`modified`, `groups_attribute`.`ticgroup_id` FROM `groups_attribute` INNER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) WHERE `groups_sampleattribute`.`sample_id` = 13 ORDER BY `groups_attribute`.`id` ASC LIMIT 1; args=(13,)
DEBUG 2017-03-28 14:30:56,649 utils.py:91 | (0.001) SELECT `groups_attribute`.`id`, `groups_attribute`.`name`, `groups_attribute`.`group_id`, `groups_attribute`.`created`, `groups_attribute`.`modified`, `groups_attribute`.`ticgroup_id` FROM `groups_attribute` INNER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) WHERE `groups_sampleattribute`.`sample_id` = 14 ORDER BY `groups_attribute`.`id` ASC LIMIT 1; args=(14,)
DEBUG 2017-03-28 14:30:56,654 utils.py:91 | (0.001) 

In [58]:
samples[0].attribute_set.first().id

DEBUG 2017-03-28 14:26:49,082 utils.py:91 | (0.001) SELECT `groups_attribute`.`id`, `groups_attribute`.`name`, `groups_attribute`.`group_id`, `groups_attribute`.`created`, `groups_attribute`.`modified`, `groups_attribute`.`ticgroup_id` FROM `groups_attribute` INNER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) WHERE `groups_sampleattribute`.`sample_id` = 13 ORDER BY `groups_attribute`.`id` ASC LIMIT 1; args=(13,)


14L

In [24]:
dataset = analysis.dataset_set.first()

DEBUG 2017-03-28 13:12:07,645 utils.py:91 | (0.003) SELECT `data_dataset`.`id`, `data_dataset`.`created`, `data_dataset`.`analysis_id` FROM `data_dataset` WHERE `data_dataset`.`analysis_id` = 2 ORDER BY `data_dataset`.`id` ASC LIMIT 1; args=(2,)


In [26]:
from compound.models import Compound

In [27]:
identified_compounds = Compound.objects.filter(identified='True', peak__dataset=dataset)

In [29]:
print identified_compounds[0]

DEBUG 2017-03-28 13:13:06,267 utils.py:91 | (0.003) SELECT `compound_compound`.`id`, `compound_compound`.`secondaryId`, `compound_compound`.`peak_id`, `compound_compound`.`formula`, `compound_compound`.`inchikey`, `compound_compound`.`ppm`, `compound_compound`.`adduct`, `compound_compound`.`identified` FROM `compound_compound` INNER JOIN `data_peak` ON ( `compound_compound`.`peak_id` = `data_peak`.`id` ) WHERE (`data_peak`.`dataset_id` = 2 AND `compound_compound`.`identified` = 'True') ORDER BY `compound_compound`.`secondaryId` ASC LIMIT 1; args=(2, 'True')


Compound object


In [30]:
test = PeakDtComparison.objects.filter(peak__compound__in=list(identified_compounds)).order_by('peak__compound__secondaryId','-peak__peakdtsample__intensity',
        'comparison').values_list('peak__compound__secondaryId','peak__id','comparison__id',
        'peak__peakdtsample__sample__id','peak__compound__id','peak__secondaryId','peak__compound__formula',
        'peak__peakdtsample__intensity','logFC','peak__peakdtsample__id').distinct()

DEBUG 2017-03-28 13:13:28,107 utils.py:91 | (0.020) SELECT `compound_compound`.`id`, `compound_compound`.`secondaryId`, `compound_compound`.`peak_id`, `compound_compound`.`formula`, `compound_compound`.`inchikey`, `compound_compound`.`ppm`, `compound_compound`.`adduct`, `compound_compound`.`identified` FROM `compound_compound` INNER JOIN `data_peak` ON ( `compound_compound`.`peak_id` = `data_peak`.`id` ) WHERE (`data_peak`.`dataset_id` = 2 AND `compound_compound`.`identified` = 'True') ORDER BY `compound_compound`.`secondaryId` ASC; args=(2, 'True')


In [31]:
list(test)

DEBUG 2017-03-28 13:13:45,225 utils.py:91 | (0.024) SELECT DISTINCT `compound_compound`.`secondaryId`, `data_peakdtcomparison`.`peak_id`, `data_peakdtcomparison`.`comparison_id`, `data_peakdtsample`.`sample_id`, `compound_compound`.`id`, `data_peak`.`secondaryId`, `compound_compound`.`formula`, `data_peakdtsample`.`intensity`, `data_peakdtcomparison`.`logFC`, `data_peakdtsample`.`id` FROM `data_peakdtcomparison` INNER JOIN `data_peak` ON ( `data_peakdtcomparison`.`peak_id` = `data_peak`.`id` ) INNER JOIN `compound_compound` ON ( `data_peak`.`id` = `compound_compound`.`peak_id` ) LEFT OUTER JOIN `data_peakdtsample` ON ( `data_peak`.`id` = `data_peakdtsample`.`peak_id` ) WHERE `compound_compound`.`id` IN (34522, 41826, 41859, 34533, 41860, 34752, 42158, 43511, 34547, 41917, 34574, 42524, 34595, 42161, 34599, 42163, 34655, 42143, 35907, 34674, 42235, 34681, 41814, 34699, 42382, 43227, 37267, 37268, 34715, 42558, 34721, 42316, 42537, 42538, 40711, 43163, 34825, 42824, 43770, 34879, 37681, 

[(4L,
  6010L,
  3L,
  17L,
  34522L,
  1L,
  u'C8H18N2O4S',
  682499136.0,
  0.175360717152522,
  72110L),
 (4L,
  6010L,
  4L,
  17L,
  34522L,
  1L,
  u'C8H18N2O4S',
  682499136.0,
  0.0866719039325616,
  72110L),
 (4L,
  6010L,
  3L,
  18L,
  34522L,
  1L,
  u'C8H18N2O4S',
  601630720.0,
  0.175360717152522,
  72116L),
 (4L,
  6010L,
  4L,
  18L,
  34522L,
  1L,
  u'C8H18N2O4S',
  601630720.0,
  0.0866719039325616,
  72116L),
 (4L,
  6010L,
  3L,
  20L,
  34522L,
  1L,
  u'C8H18N2O4S',
  593305280.0,
  0.175360717152522,
  72111L),
 (4L,
  6010L,
  4L,
  20L,
  34522L,
  1L,
  u'C8H18N2O4S',
  593305280.0,
  0.0866719039325616,
  72111L),
 (4L,
  6010L,
  3L,
  21L,
  34522L,
  1L,
  u'C8H18N2O4S',
  582681536.0,
  0.175360717152522,
  72117L),
 (4L,
  6010L,
  4L,
  21L,
  34522L,
  1L,
  u'C8H18N2O4S',
  582681536.0,
  0.0866719039325616,
  72117L),
 (4L,
  6010L,
  3L,
  16L,
  34522L,
  1L,
  u'C8H18N2O4S',
  546795584.0,
  0.175360717152522,
  72113L),
 (4L,
  6010L,
  4L,
  1

In [32]:
print len(test)

1908


In [33]:
i = 0

In [34]:
test[i]

(4L,
 6010L,
 3L,
 17L,
 34522L,
 1L,
 u'C8H18N2O4S',
 682499136.0,
 0.175360717152522,
 72110L)

In [38]:
identified_compound_pathway_list = identified_compounds.order_by("secondaryId").values_list("secondaryId", "compoundpathway__pathway__pathway__name").distinct()
pathway_super_pathway_list = Pathway.objects.all().values_list("name","datasourcesuperpathway__super_pathway__name")

superpathway_dict = {}

for pathway_name in pathway_super_pathway_list:
    if pathway_name[0] in superpathway_dict:
        if pathway_name[1] not in superpathway_dict[pathway_name[0]]:
            superpathway_dict[pathway_name[0]] = " ".join([superpathway_dict[pathway_name[0]], pathway_name[1]])
    else:
        if pathway_name[1] == None:
            superpathway_dict[pathway_name[0]] = "None"
        else:
            superpathway_dict[pathway_name[0]] = pathway_name[1]

pathway_name_dict = {}
# Create dictionnary with pathway names for each compound
for pathway_name in identified_compound_pathway_list:
    #
    if pathway_name[0] in pathway_name_dict:
        if pathway_name[1] not in pathway_name_dict[pathway_name[0]][0]:
            pathway_name_dict[pathway_name[0]][0] = " ".join([pathway_name_dict[pathway_name[0]][0], pathway_name[1]])
            pathway_name_dict[pathway_name[0]][1] = " ".join([pathway_name_dict[pathway_name[0]][1], superpathway_dict[pathway_name[1]]])
    else:
        if pathway_name[1] == None:
            pathway_name_dict[pathway_name[0]] = ["None","None"]
        else:
            pathway_name_dict[pathway_name[0]] = [pathway_name[1],  superpathway_dict[pathway_name[1]]]

DEBUG 2017-03-28 13:32:56,171 utils.py:91 | (0.008) SELECT `compound_pathway`.`name`, `compound_superpathway`.`name` FROM `compound_pathway` LEFT OUTER JOIN `compound_datasourcesuperpathway` ON ( `compound_pathway`.`id` = `compound_datasourcesuperpathway`.`pathway_id` ) LEFT OUTER JOIN `compound_superpathway` ON ( `compound_datasourcesuperpathway`.`super_pathway_id` = `compound_superpathway`.`id` ); args=()
DEBUG 2017-03-28 13:32:56,202 utils.py:91 | (0.027) SELECT DISTINCT `compound_compound`.`secondaryId`, `compound_pathway`.`name` FROM `compound_compound` INNER JOIN `data_peak` ON ( `compound_compound`.`peak_id` = `data_peak`.`id` ) LEFT OUTER JOIN `compound_compoundpathway` ON ( `compound_compound`.`id` = `compound_compoundpathway`.`compound_id` ) LEFT OUTER JOIN `compound_datasourcesuperpathway` ON ( `compound_compoundpathway`.`pathway_id` = `compound_datasourcesuperpathway`.`id` ) LEFT OUTER JOIN `compound_pathway` ON ( `compound_datasourcesuperpathway`.`pathway_id` = `compound_p

In [40]:
def has_frank_annotation(peak_id):
    p2fs = PimpFrankPeakLink.objects.filter(pimp_peak__id = peak_id).select_related(
                                'frank_peak__preferred_candidate_annotation')
    found = False
    for p2f in p2fs:
        if p2f.frank_peak.preferred_candidate_annotation:
            found = True
            break
    return found

In [66]:
data = []

list(test)
current_compound = None
current_peak = None
comparison_id_list = None
sample_id_list = None
first = True

for i in range(len(test)):
    if test[i][0] != current_compound:
        if not first:
            id_status = 'identified+fragment' if has_frank_annotation(current_peak) else 'identified'
            c_data = [compound_id,peak_secondary_id,compound_name,formula,super_pathway_names,pathway_names] + intensity_list + logfc_list + [id_status]
            data.append(c_data)
        else:
            first = False
        current_compound = test[i][0]
        current_peak = test[i][1]
        comparison_id_list = [test[i][2]]
        sample_id_list = [test[i][3]]
        compound_id = test[i][4]
        peak_secondary_id = test[i][5]
        formula = test[i][6]
        logfc_list = [round(test[i][8], 2)]
        intensity_list = [None] * len(sample_map)
        intensity_list[sample_map.index(test[i][3])] = round(test[i][7], 2)
        # Get the compound name
        compound_name = identified_compounds.get(pk=compound_id).repositorycompound_set.filter(db_name='stds_db').values_list('compound_name',flat=True).first()
        pathway_names = pathway_name_dict[current_compound][0]
        super_pathway_names = pathway_name_dict[current_compound][1]
    else:
        if current_peak == test[i][1]:
            if test[i][2] not in comparison_id_list:
                logfc_list.append(round(test[i][8], 2))
                comparison_id_list.append(test[i][2])
            if test[i][3] not in sample_id_list:
                intensity_list[sample_map.index(test[i][3])] = round(test[i][7], 2)
                sample_id_list.append(test[i][3])
                
if not first:
    id_status = 'identified+fragment' if has_frank_annotation(current_peak) else 'identified'
    c_data = [compound_id,peak_secondary_id,compound_name,formula,super_pathway_names,pathway_names] + intensity_list + logfc_list + [id_status]
    data.append(c_data)

DEBUG 2017-03-28 14:31:13,619 utils.py:91 | (0.001) SELECT `compound_compound`.`id`, `compound_compound`.`secondaryId`, `compound_compound`.`peak_id`, `compound_compound`.`formula`, `compound_compound`.`inchikey`, `compound_compound`.`ppm`, `compound_compound`.`adduct`, `compound_compound`.`identified` FROM `compound_compound` INNER JOIN `data_peak` ON ( `compound_compound`.`peak_id` = `data_peak`.`id` ) WHERE (`data_peak`.`dataset_id` = 2 AND `compound_compound`.`identified` = 'True' AND `compound_compound`.`id` = 34522) LIMIT 21; args=(2, 'True', 34522)
DEBUG 2017-03-28 14:31:13,622 utils.py:91 | (0.001) SELECT `compound_repositorycompound`.`compound_name` FROM `compound_repositorycompound` WHERE (`compound_repositorycompound`.`compound_id` = 34522 AND `compound_repositorycompound`.`db_name` = 'stds_db') ORDER BY `compound_repositorycompound`.`db_name` ASC LIMIT 1; args=(34522, 'stds_db')
DEBUG 2017-03-28 14:31:13,626 utils.py:91 | (0.001) SELECT `frank_pimpfrankpeaklink`.`id`, `fran

In [67]:
for row in data:
    if row[2] == 'L-Proline':
        print row
        break

[34574L, 23L, u'L-Proline', u'C5H9NO2', u'Metabolism; Amino acid metabolism Metabolism; Biosynthesis of other secondary metabolites Genetic Information Processing; Translation Organismal Systems; Digestive system Organismal Systems; Digestive system Environmental Information Processing; Membrane transport', u'Arginine and proline metabolism Novobiocin biosynthesis Aminoacyl-tRNA biosynthesis Mineral absorption Protein digestion and absorption ABC transporters', 308732544.0, 284501536.0, 355943616.0, 304577856.0, 510004352.0, 360450944.0, 301341312.0, 455752192.0, 382527904.0, 0.28, 0.25, 'identified']
